<a href="https://colab.research.google.com/github/21cs076/Detection-of-Landslides/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics==8.3.72

In [ ]:
from ultralytics import YOLO
from google.colab import files

model = YOLO("yolo11s.pt")
model.train(
    data = "/content/drive/MyDrive/Landslide_Dataset/data.yaml",  # Path to your data.yaml file
    epochs=80,  # Number of epochs (adjust based on your needs)
    batch=16,   # Batch size (adjust based on GPU memory), (16), 32, 64, 128
    imgsz=640,  # Image size for training ((640 is a good choice), 736)
    project = "/content/drive/MyDrive/Landslide_Dataset",  # Directory to save training results
    name="runs",  # Name for the training run
    exist_ok=True,  # Overwrite existing results if any
    save_period=10         # Save the model every 10 epochs
)
files.download("/content/drive/MyDrive/Landslide_Dataset/runs/weights/best.pt")

In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/Landslide_Dataset/runs/weights/best.pt")
#model = YOLO("/content/drive/MyDrive/models/lbest70m.pt")

tuned_conf = 0.25  # Increase confidence threshold to reduce false positives
tuned_iou  = 0.45  # Adjust NMS IoU threshold to control box suppression behavior

metrics = model.val(data="/content/drive/MyDrive/Landslide_Dataset/data.yaml", split='test')
#metrics = model.val(data="/content/drive/MyDrive/Landslide_Dataset/data.yaml", split='test', conf=tuned_conf, iou=tuned_iou)

print(f"mAP50: {metrics.box.map50}")  # Average Precision at IoU 0.5
print(f"mAP75: {metrics.box.map75}")  # Average Precision at IoU 0.75

In [ ]:
from google.colab import drive
drive.mount('/content/drive')